In [1]:
# Import Libraries
import pandas as pd

In [2]:
# Load alpha diversity metrics from QIIME2
# directories are going to be complicated -- I downloaded, unzipped, renamed, and moved everything to the main directory

shannon = pd.read_csv('shannon-alpha-diversity.tsv', sep='\t')
shannon.columns = ['sample-id','shannon_entropy']

faith = pd.read_csv('faith-alpha-diversity.tsv', sep='\t')
faith.columns = ['sample-id','faith_diversity']

pielou = pd.read_csv('pielou-alpha-diversity.tsv', sep='\t')
pielou.columns = ['sample-id','pielou_evenness']

observed = pd.read_csv('observed-features-alpha-diversity.tsv', sep='\t')
observed.columns = ['sample-id','observed_features']

# Load metadata from QIIME2 and append diversity metrics

metadata = pd.read_csv('metadata.tsv', sep='\t')
metadata = metadata.drop(metadata.index[0])
metadata = metadata.reset_index(drop=True)
metadata = metadata.merge(shannon,how='outer')
metadata = metadata.merge(faith,how='outer')
metadata = metadata.merge(pielou,how='outer')
metadata = metadata.merge(observed,how='outer')

In [ ]:
# Alpha Diversity -- Initial Shannon Boxplot
# is this something we want for the data as a whole?
fig = px.box(metadata, y="shannon_entropy")
fig.show()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df = metadata

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("x-axis:"),
    dcc.Checklist(
        id='x-axis', 
        options=[{'value': x, 'label': x} 
                 for x in ['body-site','year','month','day','subject', 'reported-antibiotic-usage','days-since-experiment-start']],
        value=['subject'], 
        labelStyle={'display': 'inline-block'}
    ),
    html.P("y-axis:"),
    dcc.RadioItems(
        id='y-axis', 
        options=[{'value': x, 'label': x} 
                 for x in ['shannon_entropy','faith_diversity','pielou_evenness','observed_features']],
        value='shannon_entropy', 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="box-plot"),
])

@app.callback(
    Output("box-plot", "figure"), 
    [Input("x-axis", "value"), 
     Input("y-axis", "value")])
def generate_chart(x, y):
    fig = px.box(df, x=x, y=y)
    return fig

app.run_server(debug=True, use_reloader=False)